# Cross validation: An Ensemble of Facebook's Prophet and Regression with ARIMA errors
---

An unweighted ensemble of Facebook Prophet with New Year holiday and Regression with new year holiday and (auto) ARIMA errors.  The ARIMA process modelled is a (1,1,3)(1,0,1,7).  This was determined by auto_arima from the `pmdarima` package.

This notebook conducts cross validation of the method using a rolling forecast origin method.

> The Reg with ARIMA error model is based on SWAST's data.  It may be prudent for each trust to fit their own automatic ARIMA model.

**The notebook outputs:**
* MASE, RMSE and MAPE at 7 day intervals from 7 to 84 days.
* 80 and 95% prediction intervals between 7 and 84 days.

These are saved into the folder `results/model_selection/stage1/`

---

# Imports

In [1]:
import pandas as pd
import numpy as np

#error measures
from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)

#models
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from fbprophet import Prophet

import warnings
warnings.filterwarnings('ignore')

In [36]:
import statsmodels as sm
import pmdarima 

print(sm.__version__)
print(pmdarima.__version__)

0.11.0
1.5.2


In [2]:
#to select exceptionally busy days as covariates.
from amb_forecast.feature_engineering import (regular_busy_calender_days)

In [3]:
#custom ensemble class
from amb_forecast.ensemble import (Ensemble, UnweightedVote)

# Data Input

The constants `TOP_LEVEL`, `STAGE`, `REGION`,`TRUST` and `METHOD` are used to control data selection and the directory for outputting results.  

> Output file is `f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv'.csv`.  where metric will be smape, rmse, mase, coverage_80 and coverage_95. Note: `REGION`: is also used to select the correct data from the input dataframe.

In [4]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'fbp-arima'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [5]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [6]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [7]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [8]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [9]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#train split into train and validation
train, val = ts_train_test_split(train, split_date=VAL_SPLIT_DATE)


In [10]:
#amount of training data
train.shape

(1279, 9)

In [11]:
#amount of validation data
val.shape

(549, 9)

# New years day

In [12]:
exceptional = regular_busy_calender_days(train[REGION], quantile=0.99)

In [13]:
new_year = pd.DataFrame({
                         'holiday': 'new_year',
                         'ds': pd.date_range(start=exceptional[0], 
                                             periods=20, 
                                             freq='YS')
                        })

In [14]:
new_year.head()

,holiday,ds
0,new_year,2013-01-01
1,new_year,2014-01-01
2,new_year,2015-01-01
3,new_year,2016-01-01
4,new_year,2017-01-01


# Wrapper classes for Prophet and statsmodels ARIMA

Adapter/wrapper classes to enable usage within `Ensemble` class and work with cross validation.

In [15]:
class FbProphetWrapper(object):
    '''
    Facade for FBProphet object - so that it can be
    used within Ensemble with methods from other packages

    '''
    def __init__(self, training_index, holidays=None, interval_width=0.8,
                 mcmc_samples=0, changepoint_prior_scale=0.05):
        self._training_index = training_index
        self._holidays = holidays
        self._interval_width = interval_width
        self._mcmc_samples = mcmc_samples
        self._cp_prior_scale = changepoint_prior_scale

    def _get_resids(self):
        return self._train - self._forecast['yhat'][:-self._h]

    def _get_preds(self):
        return self._forecast['yhat'][:-self._h].to_numpy()

    def fit(self, train):
        
        self._model = Prophet(holidays=self._holidays, 
                              interval_width=self._interval_width,
                              mcmc_samples=self._mcmc_samples,
                              changepoint_prior_scale=self._cp_prior_scale,
                              daily_seasonality=False)
        
        
        self._model.fit(self._pre_process_training(train))
        self._t = len(train)
        self._train = train
        self.predict(len(train))

    def _pre_process_training(self, train):

        if len(train.shape) > 1:
            y_train = train[:, 0]
        else:
            y_train = train

        y_train = np.asarray(y_train)
            
        #extend the training index
        if len(y_train) > len(self._training_index):
            self._training_index = pd.date_range(start=self._training_index[0], 
                                                 periods=len(y_train),
                                                 freq=self._training_index.freq)
        
        
        prophet_train = pd.DataFrame(self._training_index)
        prophet_train['y'] = y_train
        prophet_train.columns = ['ds', 'y']
        
        return prophet_train

    def predict(self, h, return_conf_int=False, alpha=0.2):
        '''
        forecast h steps ahead.
        
        Params:
        ------
        h: int
            h-step forecast
        
        return_conf_int: bool, optional (default=False)
            return 1 - alpha PI
        
        alpha: float, optional (default=0.2)
            return 1 - alpha PI
                       
        Returns:
        -------
        np.array
            If return_conf_int = False returns preds only
            
        np.array, np.array
            If return_conf_int = True returns tuple of preds, pred_ints
        '''
        if isinstance(h, (np.ndarray, pd.DataFrame)):
            h = len(h)
        
        self._h = h
        future = self._model.make_future_dataframe(periods=h)
        self._forecast = self._model.predict(future)

        if return_conf_int:
            return (self._forecast['yhat'][-h:].to_numpy(), 
                    self._forecast[['yhat_lower', 'yhat_upper']][-h:].to_numpy())
        else:
            return self._forecast['yhat'][-h:].to_numpy()
            

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)

In [16]:
class StateSpaceARIMA(object):
    '''
    Facade for statsmodels.statespace
    '''
    def __init__(self, order, seasonal_order, training_index, holidays=None):
        self._order = order
        self._seasonal_order = seasonal_order
        self._training_index = training_index
        self._holidays = holidays

    def _get_resids(self):
        return self._fitted.resid

    def _get_preds(self):
        return self._fitted.fittedvalues
    
    def _encode_holidays(self, holidays, idx):
        dummy = idx.isin(holidays).astype(int)
        dummy = pd.DataFrame(dummy)
        dummy.columns = ['holiday']
        dummy.index = idx
        return dummy

    def fit(self, y_train):
        
        #extend training index
        if len(y_train) > len(self._training_index):

            self._training_index = pd.date_range(start=self._training_index[0], 
                                                 periods=len(y_train),
                                                 freq=self._training_index.freq)
            
        holiday_train = None
        if not self._holidays is None:
            holiday_train = self._encode_holidays(self._holidays, 
                                                  self._training_index)
    
        
        self._model = ARIMA(endog=y_train,
                            exog=holiday_train,
                            order=self._order, 
                            seasonal_order=self._seasonal_order)#,
                            #enforce_stationarity=False)
        
        self._fitted = self._model.fit()
        self._t = len(train)
        
    
    def predict(self, horizon, return_conf_int=False, alpha=0.2):
        '''
        forecast h steps ahead.
        
        Params:
        ------
        h: int
            h-step forecast
        
        return_conf_int: bool, optional (default=False)
            return 1 - alpha PI
        
        alpha: float, optional (default=0.2)
            return 1 - alpha PI
                       
        Returns:
        -------
        np.array
            If return_conf_int = False returns preds only
            
        np.array, np.array
            If return_conf_int = True returns tuple of preds, pred_ints
        '''
        
        #+1 to date range then trim off the first value

        f_idx = pd.date_range(start=self._training_index[-1], 
                              periods=horizon+1,
                              freq=self._training_index.freq)[1:]
        
        #encode holidays if included.
        exog_holiday = None
        if not self._holidays is None:
            exog_holiday = self._encode_holidays(self._holidays, f_idx)
        
    
        forecast = self._fitted.get_forecast(horizon, exog=exog_holiday)
        mean_forecast = forecast.summary_frame()['mean'].to_numpy()
        
        if return_conf_int:
            df = forecast.summary_frame(alpha=alpha)
            pi = df[['mean_ci_lower', 'mean_ci_upper']].to_numpy()
            return mean_forecast, pi
            
        
        else:
            return mean_forecast

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)  

# Example of fitting the ensemble
1. Regression with New Year Holiday and Auto ARIMA errors
2. FBProphet with new years day holiday.

The code below demonstrates how to fit the model.

In [17]:
model_1 = StateSpaceARIMA(order=(1,1,3), seasonal_order=(1,0,1,7), 
                          training_index=train.index,
                          holidays=new_year['ds'].tolist())

model_2 = FbProphetWrapper(training_index=train.index, 
                           holidays=new_year)

In [18]:
estimators = {'arima': model_1, 'fbp': model_2}
ens = Ensemble(estimators, UnweightedVote())

In [19]:
#fit to training data in chosen region
ens.fit(train[REGION])

In [20]:
#predict 7 days ahead
H = 7
ens_preds = ens.predict(horizon=H)

In [21]:
#view predictions
ens_preds

array([2290.40779838, 2263.27772752, 2164.9636996 , 2090.13940692,
       2085.71037951, 2103.43703937, 2150.59791813])

In [22]:
#with prediction intervals
ens_preds, pi = ens.predict(horizon=H, return_conf_int=True)

In [23]:
ens_preds

array([2290.40779838, 2263.27772752, 2164.9636996 , 2090.13940692,
       2085.71037951, 2103.43703937, 2150.59791813])

In [24]:
pi

array([[2192.85853961, 2384.46925144],
       [2166.41304197, 2363.10771756],
       [2067.46956434, 2266.16699598],
       [1987.13386077, 2193.06241133],
       [1982.74950146, 2191.56068841],
       [1997.99161196, 2206.46546462],
       [2045.93987287, 2251.79056048]])

# Cross validation functions

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).

In [25]:
def time_series_cv(model, train, val, horizons, alpha=0.2, step=1):
    '''
    Time series cross validation across multiple horizons for a single model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Parameters:
    --------
    model - forecasting model

    train - np.array - vector of training data

    val - np.array - vector of validation data

    horizon - list of ints, forecast horizon e.g. [7, 14, 28] days
    
    alpha - float, optional (default=0.2)
        1 - alpha prediction interval specification

    step -- int, optional (default=1)
            step taken in cross validation 
            e.g. 1 in next cross validation training data includes next point 
            from the validation set.
            e.g. 7 in the next cross validation training data includes next 7 points
            (default=1)
            
    Returns:
    -------
    np.array, np.array, np.array
        - cv_preds, cv_test, cv_intervals
    '''
    
    #point forecasts
    cv_preds = [] 
    #ground truth observations
    cv_actuals = [] 
    #prediction intervals
    cv_pis = []
    
    split = 0

    print('split => ', end="")
    for i in range(0, len(val) - max(horizons) + 1, step):
        split += 1
        print(f'{split}, ', end="")
                
        train_cv = np.concatenate([train, val[:i]], axis=0)
        model.fit(train_cv)
        
        #predict the maximum horizon 
        preds, pis = model.predict(horizon=len(val[i:i+max(horizons)]), 
                                   return_conf_int=True,
                                   alpha=alpha)        
        cv_h_preds = []
        cv_test = []
        cv_h_pis = []
        
        #sub horizon calculations
        for h in horizons:
            #store the h-step prediction
            cv_h_preds.append(preds[:h])
            #store the h-step actual value
            cv_test.append(val.iloc[i:i+h])    
            cv_h_pis.append(pis[:h])
                     
        cv_preds.append(cv_h_preds)
        cv_actuals.append(cv_test)
        cv_pis.append(cv_h_pis)
        
    print('done.\n')        
    return cv_preds, cv_actuals, cv_pis

## Custom functions for calculating CV scores for point predictions and coverage.

These functions have been written to work with the output of `time_series_cv`

In [26]:
def split_cv_error(cv_preds, cv_test, error_func):
    '''
    Forecast error in the current split
    
    Params:
    -----
    cv_preds, np.array
        Split predictions
        
    
    cv_test: np.array
        acutal ground truth observations
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
        np.ndarray
            cross validation errors for split
    '''
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    '''
    Forecast errors by forecast horizon
    
    Params:
    ------
    cv_preds: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    cv_test: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
    np.ndarray
        
    '''
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [27]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

In [28]:
def get_ensemble(meta_learner=None, fb_interval=0.8):
    '''
    Create ensemble model
    '''
    if meta_learner is None:
        meta_learner = UnweightedVote()
        
    model_1 = StateSpaceARIMA(order=(1,1,3), seasonal_order=(1,0,1,7), 
                          training_index=train.index,
                          holidays=new_year['ds'].tolist())
        
    model_2 = FbProphetWrapper(training_index=train.index, 
                           holidays=new_year, interval_width=fb_interval)
    
    estimators = {'arima': model_1, 'fbp': model_2}
    return Ensemble(estimators, UnweightedVote())
    

# Run cross validation

This is run twices once each for 80 and 95% prediction intervals.  The 2nd run is required due to the way Prophet generates prediction intervals.

In [29]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
model = get_ensemble()

results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.2, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



# symmetric MAPE results

In [37]:
cv_preds, cv_test, cv_intervals = results
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,2.888748,3.069071,3.142560,3.181019,3.243412,3.307821,3.363752,3.419441,3.480343,3.522194,3.563835,3.628557,4.623541
std,1.154105,1.078228,0.854233,0.628019,0.546131,0.517146,0.488710,0.440075,0.420162,0.391239,0.393037,0.456654,1.178746
min,1.140628,1.545419,2.008209,2.256095,2.472786,2.592505,2.569449,2.657952,2.781615,2.987487,2.949513,2.996825,3.301535
25%,1.940003,2.344569,2.478941,2.669560,2.780674,2.965035,3.008267,3.111241,3.252026,3.207608,3.217273,3.211276,3.685045
50%,2.542677,2.782540,2.902718,3.019332,3.047905,3.221637,3.363829,3.410275,3.329360,3.411234,3.493673,3.659688,4.564660
75%,3.678402,3.799912,3.474135,3.672439,3.771205,3.690691,3.701352,3.677538,3.761878,3.875395,3.882196,3.908651,4.901565
max,5.458276,5.681857,4.935363,4.519707,4.289216,4.355061,4.352985,4.357056,4.377883,4.263073,4.380400,4.525349,8.945251


In [38]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/temp/Trust-fbp-arima_smape.csv


# RMSE results

In [39]:
cv_preds, cv_test, cv_intervals = results
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,76.322438,82.961792,87.044831,89.581903,92.174299,94.456973,96.489349,98.388413,100.425831,101.982125,103.442803,105.590240,129.442085
std,31.861061,31.404482,27.738723,24.119081,22.037625,20.510388,19.224001,17.644049,15.985038,13.918642,12.484009,12.640840,27.763381
min,33.998821,42.558074,55.175184,60.135793,66.126028,70.221792,71.485611,73.468009,74.251548,79.835318,79.328734,84.795769,99.052419
25%,52.443934,61.348069,69.145384,72.332606,75.969840,77.221799,79.501797,83.323549,87.298849,90.590713,93.818806,98.453261,107.556015
50%,70.422038,75.456617,79.419141,82.929364,83.921992,86.082014,98.161349,100.391861,101.743252,104.113577,103.966390,105.484323,128.492144
75%,90.927909,93.388844,91.505616,101.538721,105.289861,109.104326,114.036727,111.945535,108.595558,110.009469,111.805180,115.665794,135.595638
max,167.012731,180.920355,158.385140,142.843182,137.490799,134.245967,131.369623,137.700572,133.089819,127.728633,124.505023,133.017921,233.149446


In [40]:
#output RMSE to file
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/temp/Trust-fbp-arima_rmse.csv


# Mean Absolute Scaled Error (MASE)

Scaled by one-step insample Seasonal Naive

In [41]:
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train[REGION])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.779735,0.829899,0.850839,0.862184,0.880112,0.898571,0.914623,0.930381,0.947574,0.959693,0.971735,0.990095,1.256874
std,0.338296,0.319189,0.258226,0.197235,0.174982,0.165175,0.155135,0.139770,0.130419,0.117955,0.112327,0.126001,0.334113
min,0.327196,0.443398,0.571509,0.636428,0.691173,0.664678,0.657033,0.679980,0.730807,0.768810,0.761729,0.818899,0.899054
25%,0.531322,0.627220,0.673955,0.719479,0.738609,0.768642,0.798454,0.817743,0.860626,0.877523,0.890573,0.886508,1.002215
50%,0.689467,0.732499,0.773683,0.786387,0.833724,0.834633,0.931070,0.950313,0.945777,0.935142,0.962758,1.004145,1.238640
75%,0.962099,0.999292,0.921281,0.961827,1.019570,1.064656,1.051246,1.018840,1.019162,1.029213,1.067611,1.043209,1.329126
max,1.692838,1.725855,1.465348,1.324131,1.226330,1.240197,1.234702,1.248874,1.234477,1.203657,1.160548,1.243871,2.510996


In [42]:
#output mase to file.
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/temp/Trust-fbp-arima_mase.csv


# 80% Prediction Interval Coverage

In [43]:
#80% PIs
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.788360,0.775132,0.776014,0.781746,0.774603,0.768078,0.764928,0.761905,0.753674,0.747090,0.740741,0.731922,0.718214
std,0.214726,0.181248,0.141517,0.109109,0.101583,0.097409,0.090275,0.079552,0.077808,0.070300,0.073154,0.085424,0.125958
min,0.285714,0.357143,0.476190,0.607143,0.542857,0.500000,0.530612,0.589286,0.555556,0.585714,0.558442,0.535714,0.293151
25%,0.642857,0.642857,0.690476,0.696429,0.714286,0.738095,0.714286,0.696429,0.706349,0.714286,0.681818,0.666667,0.657534
50%,0.857143,0.857143,0.809524,0.785714,0.800000,0.785714,0.775510,0.785714,0.777778,0.757143,0.753247,0.761905,0.679452
75%,1.000000,0.892857,0.880952,0.857143,0.857143,0.821429,0.816327,0.803571,0.809524,0.800000,0.792208,0.809524,0.830137
max,1.000000,1.000000,1.000000,0.964286,0.914286,0.904762,0.918367,0.928571,0.920635,0.842857,0.844156,0.845238,0.879452


In [44]:
#write 80% coverage to file
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/temp/Trust-fbp-arima_coverage_80.csv


# 95% Prediction Interval Coverage

Rerun analysis and obtain 95% Prediction intervals

In [45]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
model = get_ensemble(fb_interval=0.95)

results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.05, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [46]:
#95% PIs
cv_preds, cv_test, cv_intervals = results
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.936508,0.928571,0.924162,0.921958,0.920635,0.919753,0.919123,0.917328,0.916520,0.916402,0.916306,0.914462,0.915068
std,0.127365,0.112066,0.096384,0.082896,0.073201,0.064414,0.056171,0.048310,0.041902,0.035814,0.032484,0.029906,0.047148
min,0.428571,0.571429,0.714286,0.750000,0.800000,0.833333,0.836735,0.857143,0.857143,0.857143,0.857143,0.869048,0.747945
25%,0.857143,0.892857,0.857143,0.892857,0.857143,0.857143,0.857143,0.875000,0.888889,0.900000,0.896104,0.898810,0.894521
50%,1.000000,1.000000,0.952381,0.964286,0.942857,0.928571,0.918367,0.892857,0.888889,0.900000,0.909091,0.916667,0.917808
75%,1.000000,1.000000,1.000000,1.000000,1.000000,0.976190,0.959184,0.964286,0.960317,0.957143,0.928571,0.916667,0.954795
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.971429,0.974026,0.976190,0.972603


In [47]:
#write 95% coverage to file
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/temp/Trust-fbp-arima_coverage_95.csv


# End